In [ ]:
import pandas as pd
import json

# Load datasets
portfolio = pd.read_json('portfolio.json', orient='records', lines=True)
profile = pd.read_json('profile.json', orient='records', lines=True)
transcript = pd.read_json('transcript.json', orient='records', lines=True)
'''
with open('portfolio.json') as f:
    portfolio = pd.DataFrame(json.load(f))

with open('profile.json') as f:
    profile = pd.DataFrame(json.load(f))

with open('transcript.json') as f:
    transcript = pd.DataFrame(json.load(f))
'''

"\nwith open('portfolio.json') as f:\n    portfolio = pd.DataFrame(json.load(f))\n\nwith open('profile.json') as f:\n    profile = pd.DataFrame(json.load(f))\n\nwith open('transcript.json') as f:\n    transcript = pd.DataFrame(json.load(f))\n"

In [ ]:
print(portfolio.shape)
print(profile.shape)
print(transcript.shape)

(10, 6)
(17000, 5)
(306534, 4)


In [ ]:
# Parse 'value' column in transcript
transcript['offer_id'] = transcript['value'].apply(lambda x: x.get('offer id') if 'offer id' in x else x.get('offer_id') if 'offer_id' in x else None)
transcript['transaction_amount'] = transcript['value'].apply(lambda x: x.get('amount') if 'amount' in x else None)
del transcript['value']

transcript.head()

,person,event,time,offer_id,transaction_amount
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN


In [ ]:
# Transform 'became_member_on' to membership duration
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')
profile['membership_duration'] = (pd.Timestamp.now() - profile['became_member_on']).dt.days

In [ ]:
profile.head()

,gender,age,id,became_member_on,income,membership_duration
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,2894
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,2741
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,2379
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,2808
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,2721


In [ ]:
# Merge datasets
# Merge portfolio and transcript on 'offer_id'
merged_data = pd.merge(transcript, portfolio, how='left', left_on='offer_id', right_on='id')

# Merge profile data into the merged_data on customer ID ('person' in transcript and 'id' in profile)
merged_data = pd.merge(merged_data, profile, how='left', left_on='person', right_on='id', suffixes=('_portfolio', '_profile'))

# Drop redundant columns
merged_data = merged_data.drop(columns=['id_portfolio', 'id_profile'])

In [ ]:
merged_data.head()

,person,event,time,offer_id,transaction_amount,reward,channels,difficulty,duration,offer_type,gender,age,became_member_on,income,membership_duration
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,5.0,"[web, email, mobile]",5.0,7.0,bogo,F,75,2017-05-09,100000.0,2808
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,5.0,"[web, email]",20.0,10.0,discount,None,118,2017-08-04,NaN,2721
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5,NaN,2.0,"[web, email, mobile]",10.0,7.0,discount,M,68,2018-04-26,70000.0,2456
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,NaN,2.0,"[web, email, mobile, social]",10.0,10.0,discount,None,118,2017-09-25,NaN,2669
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,10.0,"[web, email, mobile, social]",10.0,5.0,bogo,None,118,2017-10-02,NaN,2662


In [ ]:
# Feature engineering
# Calculate total spend and transaction count per customer
customer_summary = merged_data[merged_data['event'] == 'transaction'].groupby('person').agg(
    total_spent=('transaction_amount', 'sum'),
    transaction_count=('transaction_amount', 'count')
).reset_index()

# Calculate offer completion rate
offer_summary = merged_data[merged_data['event'] == 'offer completed'].groupby('person').agg(
    offers_completed=('event', 'count')
).reset_index()

In [ ]:
customer_summary.head()

,person,total_spent,transaction_count
0,0009655768c64bdeb2e877511632db8f,127.60,8
1,00116118485d4dfda04fdbaba9a87b5c,4.09,3
2,0011e0d4e6b944f998e987f904e8c1e5,79.46,5
3,0020c2b971eb4e9188eac86d93036a77,196.86,8
4,0020ccbbb6d84e358d3414a3ff76cffd,154.05,12


In [ ]:
offer_summary.head()

,person,offers_completed
0,0009655768c64bdeb2e877511632db8f,3
1,0011e0d4e6b944f998e987f904e8c1e5,3
2,0020c2b971eb4e9188eac86d93036a77,3
3,0020ccbbb6d84e358d3414a3ff76cffd,3
4,003d66b6608740288d6cc97a6903f4f0,3


In [ ]:
# Merge customer and offer summaries
customer_data = pd.merge(customer_summary, offer_summary, on='person', how='left').fillna(0)

# Add demographic features
customer_data = pd.merge(customer_data, profile[['id', 'age', 'income', 'gender', 'membership_duration']],
                         left_on='person', right_on='id', how='left').drop(columns=['id'])

# Categorize customers into high-value and others
customer_data['customer_tier'] = pd.qcut(customer_data['total_spent'], q=4,
                                         labels=['Low Value', 'Mid Value', 'High Value', 'Top Value'])

# Output customer_data
customer_data.head()

,person,total_spent,transaction_count,offers_completed,age,income,gender,membership_duration,customer_tier
0,0009655768c64bdeb2e877511632db8f,127.60,8,3.0,33,72000.0,M,2826,High Value
1,00116118485d4dfda04fdbaba9a87b5c,4.09,3,0.0,118,NaN,None,2457,Low Value
2,0011e0d4e6b944f998e987f904e8c1e5,79.46,5,3.0,40,57000.0,O,2563,High Value
3,0020c2b971eb4e9188eac86d93036a77,196.86,8,3.0,59,90000.0,F,3239,Top Value
4,0020ccbbb6d84e358d3414a3ff76cffd,154.05,12,3.0,24,60000.0,F,2987,Top Value


In [ ]:
# Save results to CSV
customer_data.to_csv('customer_loyalty_analysis.csv', index=False)